In [1]:
import pandas as pd
from collections import defaultdict
import networkx as nx
import pickle
import collections
import math
from tqdm import tqdm
import numpy as np
from collections import defaultdict
import statistics
import operator

In [2]:
source_destination=pd.read_csv('wiki-topcats-reduced.txt',sep="\t",header=None,names=["source","destination"])

In [3]:
source_destination_dict = pickle.load(open('source_destination_dict.p', 'rb'))

In [4]:
source_destination_tuples=[(row["source"],row["destination"]) for idx,row in tqdm(source_destination.iterrows())]

2645247it [01:49, 24223.40it/s]


In [5]:
Gtup_directed = nx.DiGraph()
Gtup_directed.add_edges_from(source_destination_tuples)
print("Number of nodes:",len(Gtup_directed.nodes()),"\nNumber of edges:",len(Gtup_directed.edges()))

Number of nodes: 461193 
Number of edges: 2645247


In [6]:
categories=pd.read_csv('wiki-topcats-categories.txt',sep="\n",header=None)

In [7]:
def num_cat(row):
    cat=row
    return len(cat.split("; ")[1].split(" "))

In [8]:
categories=categories[categories.iloc[:,0].apply(lambda x:True if num_cat(x)>3500 else False)]

In [9]:
categories_dict=defaultdict(list)
for idx in categories.index:
    cat_and_values=categories[0].loc[idx].split("; ")
    cat_name=cat_and_values[0].split(":")[1]
    categories_dict[cat_name]=list(map(int,cat_and_values[1].split(" ")))

In [10]:
categories = {}
all_nodes=set(Gtup_directed.nodes())
for key, values in categories_dict.items():
        categories[key] = all_nodes.intersection(set(values))
        if len(categories[key]) < 3500:
            del(categories[key])

In [11]:
import random
cat_nodes_lst = random.sample(categories['Windows_games'], 100)
Gtup_directed_sample_C0=Gtup_directed.subgraph(cat_nodes_lst)

In [12]:
nx.info(Gtup_directed_sample_C0)

'Name: \nType: SubDiGraph\nNumber of nodes: 100\nNumber of edges: 4\nAverage in degree:   0.0400\nAverage out degree:   0.0400'

In [12]:
def bfs_shortest_path(graph, start):
    explored = set()
    queue = [start]
    shortest_path_dict = {}
    counter_level = 0
    to_visit_list = []
    while queue:
        for node in queue:
            if node not in explored:
                shortest_path_dict[node] = counter_level
                neighbours = graph[node]
                explored.add(node)
                to_visit_list.extend(neighbours)
        queue = []
        queue.extend(to_visit_list)
        to_visit_list = []
        counter_level += 1
    return shortest_path_dict

In [13]:
l = []
for article in tqdm(Gtup_directed_sample_C0.nodes()):
    l.append(bfs_shortest_path(Gtup_directed, article))   


100%|██████████| 100/100 [03:10<00:00,  1.69s/it]


In [63]:
d = defaultdict(list)
for node in l:
    for key, value in node.items():
        d[key].append(value)

In [73]:
z = defaultdict(list)
for key, value in d.items():
    z[key] = np.min(d[key])

In [97]:
categories_edited = categories.copy()

In [127]:
to_delete_article = []
for article in categories_edited['Windows_games']:
    if article not in cat_nodes_lst:
        to_delete_article.append(article)

In [128]:
for article in to_delete_article:
    categories_edited['Windows_games'].remove(article)
    

In [129]:
len(categories_edited['Windows_games'])

100

Ok, now 'Windows_games' has 100 article

In [130]:
cat_inv_dic=defaultdict(list)
for cat_k,cat_v in tqdm(categories_edited.items()):
    for node,val in z.items():
        if node in cat_v:
            cat_inv_dic[cat_k].append(val)

100%|██████████| 29/29 [00:03<00:00,  8.88it/s]


In [135]:
for cat_k in categories_edited.keys():
    cat_inv_dic[cat_k].extend([100]*(len(categories_edited[cat_k])-len(cat_inv_dic[cat_k])))

In [137]:
for key, value in cat_inv_dic.items():
    cat_inv_dic[key] = statistics.median(cat_inv_dic[key])

In [138]:
cat_inv_dic

defaultdict(list,
            {'English_footballers': 6.0,
             'The_Football_League_players': 6.0,
             'Association_football_forwards': 7,
             'Association_football_goalkeepers': 100,
             'Association_football_midfielders': 100,
             'Association_football_defenders': 11.0,
             'Living_people': 6.0,
             'Harvard_University_alumni': 6,
             'Major_League_Baseball_pitchers': 6.0,
             'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 5,
             'Indian_films': 5.0,
             'Year_of_death_missing': 100.0,
             'Year_of_birth_missing_(living_people)': 6.0,
             'Rivers_of_Romania': 6,
             'Main_Belt_asteroids': 100.0,
             'Asteroids_named_for_people': 100,
             'English-language_albums': 5.0,
             'British_films': 4.0,
             'English-language_films': 4,
             'American_films': 4,
             'People_from_New_York_City':

In [140]:
cat_inv_dic_rank= sorted(cat_inv_dic.items(), key=operator.itemgetter(1), reverse= False)

In [141]:
cat_inv_dic_rank

[('Windows_games', 0.0),
 ('British_films', 4.0),
 ('English-language_films', 4),
 ('American_films', 4),
 ('American_television_actors', 4),
 ('American_film_actors', 4),
 ('Black-and-white_films', 4),
 ('People_from_New_York_City', 4.5),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 5),
 ('Indian_films', 5.0),
 ('English-language_albums', 5.0),
 ('Debut_albums', 5),
 ('American_military_personnel_of_World_War_II', 5.0),
 ('English_footballers', 6.0),
 ('The_Football_League_players', 6.0),
 ('Living_people', 6.0),
 ('Harvard_University_alumni', 6),
 ('Major_League_Baseball_pitchers', 6.0),
 ('Year_of_birth_missing_(living_people)', 6.0),
 ('Rivers_of_Romania', 6),
 ('Place_of_birth_missing_(living_people)', 6.0),
 ('Association_football_forwards', 7),
 ('Association_football_defenders', 11.0),
 ('Association_football_goalkeepers', 100),
 ('Association_football_midfielders', 100),
 ('Year_of_death_missing', 100.0),
 ('Main_Belt_asteroids', 100.0),
 ('Asteroid

Step1

i wrote two procedure:
one,very ugly, a lot of mechanical passages (to complete)
and one other (it is at the end of the code), more linear--> there is a function to complete

In [397]:
H = Gtup_directed.subgraph(categories_dict['Windows_games'])

In [398]:
nx.info(H)

'Name: \nType: SubDiGraph\nNumber of nodes: 4025\nNumber of edges: 6117\nAverage in degree:   1.5198\nAverage out degree:   1.5198'

In [464]:
in_edge_dict0 = defaultdict(int)
for tupla in list(H.edges()):
    if tupla[1] in categories_dict['Windows_games']:
        in_edge_dict0[tupla[1]] += 1

In [466]:
sorted_in_edge_dict= sorted(in_edge_dict0.items(), key=operator.itemgetter(1), reverse= True)

In [ ]:
sorted_in_edge_dict

In [468]:
cat_inv_dic_rank[1]

('British_films', 4.0)

In [499]:
def in_edge_dict(graph, list_article, predecessors_weight_dict):
    in_edge_dict = defaultdict(int)
    for tupla in list(graph.edges()):
        try:
            if tupla[1] in list_article:
                if tupla[0] in list_article:
                    source=predecessors_weight_dict[tupla[0]]
                else:
                    source=predecessors_weight_dict[tupla[0]]
                    in_edge_dict[tupla[1]] += source
        except:
            in_edge_dict[tupla[1]] = 0
   
    return in_edge_dict

In [500]:
list_of_nodes = []
for article in categories_dict['Windows_games']:
    list_of_nodes.append(article)

In [501]:
for article in categories_edited['British_films']:
     list_of_nodes.append(article)

In [502]:
H1 = Gtup_directed.subgraph(list_of_nodes)

In [503]:
in_edge_dict1 = in_edge_dict(H1,categories_edited['British_films'], in_edge_dict0)

In [504]:
sorted_in_edge_dict1= sorted(in_edge_dict1.items(), key=operator.itemgetter(1), reverse= True)

In [505]:
cat_inv_dic_rank[2]

('English-language_films', 4)

In [506]:
for article in categories_dict['English-language_films']:
    list_of_nodes.append(article)

In [507]:
H2 = Gtup_directed.subgraph(list_of_nodes)

In [ ]:
here we've to found the way to create a dictionary with in_edge_dict1 and in_edge_dict0


In [508]:
in_edge_dict2 = in_edge_dict(H2,categories_edited['English-language_films'], dictionary_to_find)

In [509]:
sorted_in_edge_dict2= sorted(in_edge_dict2.items(), key=operator.itemgetter(1), reverse= True)

In [426]:
for article in categories_dict['American_films']:
    list_of_nodes.append(article)

In [427]:
H3 = Gtup_directed.subgraph(list_of_nodes)

In [ ]:
here we've to found the way to create a dictionary with in_edge_dict1 and in_edge_dict0 and in_edge_dict3

In [ ]:
in_edge_dict3 = in_edge_dict(H3,categories_dict['American_films'], dictionary_to_found)

In [ ]:
sorted_in_edge_dict3= sorted(in_edge_dict3.items(), key=operator.itemgetter(1), reverse= True)

In [ ]:
for article in categories_dict['American_television_actors']:
    list_of_nodes.append(article)

In [ ]:
H4 = Gtup_directed.subgraph(list_of_nodes)

In [ ]:
in_edge_dict4 = in_edge_dict(H4,categories_dict['American_television_actors'], in_edge_dict3)

In [ ]:
sorted_in_edge_dict4= sorted(in_edge_dict4.items(), key=operator.itemgetter(1), reverse= True)

The second way (i prefer this)
here there is the same problem that we've already met: update each time a dictionary with the dictionary_weight of the precedent category. Finally, we have to complete the request 

In [550]:
for article in categories_dict['Windows_games']:
    list_of_nodes.append(article)

In [551]:
H = Gtup_directed.subgraph(list_of_nodes)

In [ ]:
in_edge_dict_precedent = defaultdict(int)
for tupla in list(H.edges()):
    if tupla[1] in categories_dict['Windows_games']:
        in_edge_dict_precedent[tupla[0]] += 1

i didn't have added this previous part in the for loop because it give me an error

In [ ]:
in_edge_dict_list = defaultdict(int)

In [ ]:
in_edge_dict_list = in_edge_dict_list.append(in_edge_dict_precedent)

In [537]:
for category in tqdm(range(0, len(cat_inv_dic_rank))):
    for article in categories_edited[cat_inv_dic_rank[category][0]]:
        list_of_nodes.append(article)
    H = Gtup_directed.subgraph(list_of_nodes)
    in_edge_dict_current = in_edge_dict(H,categories_edited[cat_inv_dic_rank[category][0]], in_edge_dict_precedent)
    in_edge_dict_list = in_edge_dict_list.append(in_edge_dict_current)
    in_edge_dict_precedent.update(in_edge_dict_current)

100%|██████████| 29/29 [03:56<00:00, 13.86s/it]
